# OpenAI gym: Copy-v0 with Simple Q-learning  
- Observation dim = [1]  
- Action dim = [2,2,5]. Use encoding 0-100
- The rest is a standard Q learning with a memory relay
- Initial exploration is random. I found 100 episodes were not enough, and I kept using 1000 episodes.

In [1]:
import datetime
print(datetime.datetime.now().isoformat())

2017-08-27T00:30:05.059532


In [2]:
import tensorflow as tf
import numpy as np
import gym
from gym import wrappers

In [3]:
tf.reset_default_graph()
k_init = tf.contrib.layers.variance_scaling_initializer()
X = tf.placeholder(tf.int32,shape=[None,1])
hid1 = tf.reshape(tf.one_hot(X,6),[-1,6])
hid2 = tf.layers.dense(hid1,64,activation=tf.nn.elu,kernel_initializer=k_init)
hid3 = tf.layers.dense(hid2,128,activation=tf.nn.elu,kernel_initializer=k_init)
Q = tf.layers.dense(hid2,20)
action_ph = tf.placeholder(tf.int32,shape=[None,])
Q_action = tf.reduce_sum(Q*tf.one_hot(action_ph,20),axis=1,keep_dims=True)
Qexpected = tf.placeholder(tf.float32,shape=[None,1])
loss = tf.losses.mean_squared_error(Qexpected,Q_action)
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001)
train_op = optimizer.minimize(loss)

In [4]:
from collections import deque
class Memory(object):
    def __init__(self,memory_size=10000):
        self.memory = deque(maxlen=memory_size)
        self.memory_size = memory_size
        
    def __len__(self):
        return len(self.memory)
    
    def append(self,item):
        self.memory.append(item)
        
    def sample_batch(self,batch_size=256):
        idx = np.random.permutation(len(self.memory))[:batch_size]
        return [self.memory[i] for i in idx]

In [5]:
gamma = 0.99
max_episode = 5000
batch_size = 256
memory_warmup = batch_size*3
memory = Memory()
explore_eps = 1000

env = gym.make('Copy-v0')
env = wrappers.Monitor(env,'./tmp/',force=True)
obs = env.reset()
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()
    iteration = 0
    episode = 0
    episode_reward = 0
    episode_steps = 0
    while episode <= max_episode:
        print('\riter {}, ep {}'.format(iteration,episode),end='')
        p = episode/explore_eps
        if np.random.rand() > p:
            action = env.action_space.sample()
        else:
            Q_val = Q.eval(feed_dict={X:np.reshape(obs,[1,-1])})
            action = np.argmax(Q_val,axis=1)
            action = np.unravel_index(action[0],[2,2,5])
        next_obs,reward,done,_ = env.step(action)
        memory.append([obs,np.ravel_multi_index(action,[2,2,5]),reward,next_obs,done])
        if iteration > memory_warmup:
            memory_batch = memory.sample_batch(batch_size)
            extract_mem = lambda k: np.array([item[k] for item in memory_batch])
            obs_batch = extract_mem(0)
            action_batch = extract_mem(1)
            reward_batch = extract_mem(2)
            next_obs_batch = extract_mem(3)
            done_batch = extract_mem(4)
            Qnext_val = Q.eval(feed_dict={X:np.expand_dims(next_obs_batch,axis=1)})
            Qnext_val = np.max(Qnext_val,axis=1,keepdims=True)
            Qexpected_batch = reward_batch + gamma*(1-done_batch)*Qnext_val.ravel()
            Qexpected_batch = np.expand_dims(Qexpected_batch,1)
            train_op.run(feed_dict={X:np.expand_dims(obs_batch,axis=1),action_ph:action_batch,Qexpected:Qexpected_batch})
        iteration += 1
        episode_reward += reward
        episode_steps += 1
        if done:
            if iteration>memory_warmup and episode%100==0:
                print(', ep reward {}, ep steps {}'.format(episode_reward, episode_steps))
            episode_reward = 0
            episode_steps = 0
            episode += 1
            obs = env.reset()
        else:
            obs = next_obs
env.close()

[2017-08-27 00:30:10,892] Making new env: Copy-v0
[2017-08-27 00:30:10,985] Clearing 18 monitor files from previous run (because force=True was provided)
[2017-08-27 00:30:10,988] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000000.json
[2017-08-27 00:30:11,094] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000001.json
[2017-08-27 00:30:11,099] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000008.json
[2017-08-27 00:30:11,114] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000027.json
[2017-08-27 00:30:11,140] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000064.json
[2017-08-27 00:30:11,173] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/open

iter 769, ep 316

[2017-08-27 00:30:11,433] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000343.json


iter 1018, ep 400, ep reward 0.5, ep steps 8
iter 1361, ep 495

[2017-08-27 00:30:12,899] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video000512.json


...

[2017-08-27 00:32:41,871] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video004000.json


iter 56556, ep 4000, ep reward 30.0, ep steps 30
iter 59677, ep 4100, ep reward 31.0, ep steps 31
iter 62776, ep 4200, ep reward 32.0, ep steps 32
iter 65881, ep 4300, ep reward 31.0, ep steps 31
iter 68980, ep 4400, ep reward 32.0, ep steps 32
iter 72094, ep 4500, ep reward 30.0, ep steps 30
iter 75179, ep 4600, ep reward 31.0, ep steps 31
iter 78282, ep 4700, ep reward 32.0, ep steps 32
iter 81372, ep 4800, ep reward 32.0, ep steps 32
iter 84463, ep 4900, ep reward 30.0, ep steps 30
iter 87530, ep 4999

[2017-08-27 00:34:07,657] Starting new video recorder writing to /Users/winter/Google Drive/handson-ml/tmp/openaigym.video.0.1673.video005000.json


iter 87562, ep 5000, ep reward 32.0, ep steps 32


[2017-08-27 00:34:07,873] Finished writing results. You can upload them to the scoreboard via gym.upload('/Users/winter/Google Drive/handson-ml/tmp')


In [6]:
gym.upload('./tmp/',api_key='sk_BlwjttPKR6ZsXVrObENYA')

[2017-08-27 00:34:07,916] [Copy-v0] Uploading 5001 episodes of training data
[2017-08-27 00:34:10,513] [Copy-v0] Uploading videos of 15 training episodes (3807 bytes)
[2017-08-27 00:34:10,859] [Copy-v0] Creating evaluation object from ./tmp/ with learning curve and training video
[2017-08-27 00:34:11,425] 
****************************************************
You successfully uploaded your evaluation on Copy-v0 to
OpenAI Gym! You can find it at:

    https://gym.openai.com/evaluations/eval_qdzwvK4RNa1Dx71JtH2g

****************************************************
